In [1]:
import numpy as np
import time
import ctypes
from utility.path_tracking import search, \
smooth, transform_points_from_image2real, \
transform2robot_frame, is_near, \
get_distance, \
transform_pos_angle, pid, \
send_path_4_drawing, \
pioneer_robot_model
from vision.tracker import Tracker, get_marker_object, get_markers, CAM_MAT, CAM_DIST
from robot.evolved_robot import EvolvedRobot
import vrep.vrep as vrep

In [2]:
vision_thread = Tracker(mid=5,
                    transform=None,
                    mid_aux=0,
                    video_source=-1,
                    capture=False,
                    show=True,
                    debug=False,
                   )
vision_thread.start()

starting Tracker, video source:  -1
Last markers len: 8
4444444444444445Restore Calibration


In [ ]:
def follow_point():
    try:
        grid = np.full((880, 1190), 255)
        lad = 0.05 # look ahead distance in meters (m)
        wheel_axis = 0.11 # wheel axis distance in meters (m)
        wheel_radius = 0.02 # wheel radius in meters (m)
        indx = 0
        theta = 0.0
        count = 0
        om_sp = 0

        d_controller   = pid(kp=0.5, ki=0, kd=0)
        omega_controller = pid(kp=0.5, ki=0, kd=0)

        robot = EvolvedRobot('thymio-II', -1, None, 1, None)

        robot_m = get_marker_object(7)
        while robot_m.realxy() is None:
            # obtain current position of the robot
            robot_m = get_marker_object(7)

        goal_m = get_marker_object(8)
        while goal_m.realxy() is None:
            # obtain goal marker position
            goal_m = get_marker_object(8)
        
        # import pdb; pdb.set_trace();

        # transform robot position to grid system
        robot_current_position = robot_m.realxy()[:2]

        # transform goal position to grid system
        goal_position = goal_m.realxy()[:2]

        # point to track
        path_to_track = np.array([goal_position])

        while not is_near(robot_current_position, goal_position, dist_thresh = 0.05):
            # get robot marker
            robot_m = get_marker_object(7)
            if robot_m.realxy() is not None:
                # update current position of the robot
                robot_current_position = robot_m.realxy()[:2]

            # calculate robot orientation
            theta = 2*np.pi - robot_m.orientation()
            theta = np.arctan2(np.sin(theta), np.cos(theta))
            
            print('r_t: ', theta)
            # path transformation to vehicle coordinates; relative to the robot
            path_transformed = transform2robot_frame(robot_current_position, path_to_track, theta)
            print('p_t', path_transformed)
            # get distance of each carrot point; relative to the robots
            dist = get_distance(path_transformed, np.array([0,0]))           

            # loop to determine which point will be the carrot/goal point
            for i in range(dist.argmin(), dist.shape[0]):
                if dist[i] < lad and indx <= i:
                    indx = i

            # orientation error relative to the robot
            orient_error = np.arctan2(path_transformed[indx,1], path_transformed[indx,0])
            print('e_t: ', orient_error, '\n')
            # PID controller; desired velocity and rotation
            v_sp = d_controller.control(dist[indx])                     
            om_sp = omega_controller.control(orient_error)
            vr, vl = pioneer_robot_model(v_sp, om_sp, wheel_axis, wheel_radius)
            robot.t_set_motors(vl*20, vr*20)
            count += 1
        else:
            print('GOAAAAAAALL !!')
            print('robot_position: ', robot_current_position)
            print('robot_goal: ', goal_position)
            robot.t_stop()   
    finally:
        time.sleep(0.1)